A self-querying retriever is one that, as the name suggests, has the ability to query itself. Specifically, given any natural language query, the retriever uses a query-constructing LLM chain to write a structured query and then applies that structured query to its underlying VectorStore. This allows the retriever to not only use the user-input query for semantic similarity comparison with the contents of stored documents but to also extract filters from the user query on the metadata of stored documents and to execute those filters.

For demonstration purposes we'll use a Chroma vector store. We've created a small demo set of documents that contain summaries of movies.

Note: The self-query retriever requires you to have lark package installed.

In [1]:
# from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name = "moka-ai/m3e-base")

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]
vectorstore = Chroma.from_documents(docs, embedding_model)

d:\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Creating our self-querying retriever
Now we can instantiate our retriever. To do this we'll need to provide some information upfront about the metadata fields that our documents support and a short description of the document contents.

In [2]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie in English"

# llm = ChatOpenAI(temperature=0)
openai_api_base_address = "http://192.168.3.84:20000/v1"
llm = ChatOpenAI(openai_api_key = "aaabbbcccdddeeefffedddsfasdfasdf", 
    openai_api_base = openai_api_base_address,
    model_name = "vicuna-7b")

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

## Testing it out
And now we can actually try using our retriever!

In [3]:
# This example only specifies a filter
retriever.invoke("I want to watch a movie rated higher than 8.5")

OutputParserException: Parsing text
I'm sorry, but the examples provided do not match the provided schema. The examples provided do not contain a "query" or "filter" field, and the values do not match the format specified in the schema.

Please provide a valid example that matches the schema, and I will be happy to help you structure your request.
 raised following error:
Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)

In [4]:
# This example specifies a query and a filter
retriever.invoke("Has Greta Gerwig directed any movies about women")

[Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'director': 'Greta Gerwig', 'rating': 8.3, 'year': 2019})]

In [5]:
# This example specifies a composite filter
retriever.invoke("What's a highly rated (above 8.5) science fiction film?")

OutputParserException: Parsing text
Great, let's apply the request schema to the examples.

Example 1:

| Query | Filter |
| --- | --- |
| teenager love |  |
| and(or(eq("artist", "Taylor Swift"), eq("artist", "Katy Perry")), lt("length", 180), eq("genre", "pop")) |

Example 2:

| Query | Filter |
| --- | --- |
|  |  |
| NO\_FILTER |

Example 3:

| Query | Filter |
| --- | --- |
| Brief summary of a movie in English |  |
| and(eq("genre", "action"), gt("year", 2018)) |

Can you please check if the response is what you expected?
 raised following error:
Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)

In [6]:
# This example specifies a query and composite filter
retriever.invoke(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

OutputParserException: Parsing text
To structure the user's query to match the provided request schema, we need to break down the query string into its component parts and create a JSON object with the appropriate values.

First, we need to parse the query string into a JSON object.
```json
{
    "query": "after 1990 and before 2005 and is animated and is about toys"
}
```
Next, we need to create a filter statement to filter the documents based on the user's query.
```json
{
    "filter": "and(or(gte(year(), 1990), lt(year(), 2005)), eq(genre(), "animated"), eq(director(), "toys"))"
}
```
Finally, we need to format the JSON object as a markdown code snippet.
```vbnet
# Structured Request

{
    "query": "after 1990 and before 2005 and is animated and is about toys",
    "filter": "and(or(gte(year(), 1990), lt(year(), 2005)), eq(genre(), \"animated\"), eq(director(), \"toys\"))"
}
```
In this example, the query string "after 1990 and before 2005 and is animated and is about toys" matche

## Filter k

We can also use the self query retriever to specify k: the number of documents to fetch.

We can do this by passing enable_limit=True to the constructor.

In [7]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
)

# This example only specifies a relevant query
retriever.invoke("What are two movies about dinosaurs")

OutputParserException: Parsing text
Here's an example of a structured request for the given user query:
```json
{
    "query": "teenager love",
    "filter": "and(or(eq(\"artist\", \"Taylor Swift\"), eq(\"artist\", \"Katy Perry\")), lt(\"length\", 180), eq(\"genre\", \"pop\"))"
}
```
In this request, the query is "teenager love" which is a text string that matches the contents of the songs, and the filter statement checks for songs by Taylor Swift or Katy Perry, with a length under 3 minutes and in the dance pop genre. The filter statement uses the comparison and logical operation statements as provided in the requested schema.
 raised following error:
Received invalid attributes artist. Allowed attributes are ['genre', 'year', 'director', 'rating']

## Constructing from scratch with LCEL

To see what's going on under the hood, and to have more custom control, we can reconstruct our retriever from scratch.

First, we need to create a query-construction chain. This chain will take a |user query and generated a StructuredQuery object which captures the filters specified by the user. We provide some helper functions for creating a prompt and output parser. These have a number of tunable params that we'll ignore here for simplicity.

In [8]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
)
output_parser = StructuredQueryOutputParser.from_components()
query_constructor = prompt | llm | output_parser

Let's look at our prompt:

In [9]:
print(prompt.format(query="dummy question"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

And what our full chain produces:

In [10]:
query_constructor.invoke(
    {
        "query": "What are some sci-fi movies from the 90's directed by Luc Besson about taxi drivers"
    }
)

OutputParserException: Parsing text
To fulfill this request, the following structured request would be sent:

{
"query": "sci-fi movies from the 90's directed by Luc Besson about taxi drivers",
"filter": "and(eq(\"genre\", \"sci-fi\"), eq(\"year\", 1990), eq(\"director\", \"Luc Besson\"), contain(\"rating\", "9.0"))"
}

Explanation:

* The query string contains the comparison statement "eq(genre,'sci-fi')" which compares the value of the "genre" attribute to the string "sci-fi".
* The filter statement contains the comparison statements "eq(year, 1990)" and "eq(director, 'Luc Besson')" which compare the values of the "year" and "director" attributes to the specified values.
* The filter statement also contains the logical operation statement "and" which combines the two comparison statements.
* The filter statement also contains the logical operator "and" which combines the two comparison statements.
* The filter statement also contains the comparison statement "contain(rating, '9.0')" 

The query constructor is the key element of the self-query retriever. To make a great retrieval system you'll need to make sure your query constructor works well. Often this requires adjusting the prompt, the examples in the prompt, the attribute descriptions, etc. For an example that walks through refining a query constructor on some hotel inventory data, check out this cookbook.

The next key element is the structured query translator. This is the object responsible for translating the generic StructuredQuery object into a metadata filter in the syntax of the vector store you're using. LangChain comes with a number of built-in translators. To see them all head to the Integrations section.

In [11]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

retriever = SelfQueryRetriever(
    query_constructor=query_constructor,
    vectorstore=vectorstore,
    structured_query_translator=ChromaTranslator(),
)

In [12]:
retriever.invoke(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

[]